In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_961636/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.8)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_3_20,0.994728,0.626256,0.985181,0.986146,0.985875,0.001795,0.221871,0.002833,0.004495,0.003664,0.018693,0.042364,0.989018,0.042704,142.645842,327.004166,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_3_21,0.994727,0.626406,0.985137,0.986080,0.985817,0.001795,0.221781,0.002842,0.004516,0.003679,0.018668,0.042368,0.989015,0.042709,142.645406,327.003730,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_3_14,0.994706,0.625167,0.986618,0.986143,0.986402,0.001802,0.222517,0.002559,0.004496,0.003527,0.019031,0.042453,0.988971,0.042794,142.637395,326.995719,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_3_15,0.994695,0.625274,0.986506,0.986072,0.986317,0.001806,0.222453,0.002580,0.004519,0.003549,0.019028,0.042500,0.988947,0.042841,142.633051,326.991375,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_3_19,0.994684,0.626054,0.985433,0.985948,0.985844,0.001810,0.221991,0.002785,0.004559,0.003672,0.018868,0.042544,0.988924,0.042886,142.628862,326.987186,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,model_1_0_4,0.548844,0.416932,0.473934,0.468019,0.472250,0.153596,0.346134,0.163449,0.161050,0.162250,0.194460,0.391913,0.060091,0.395061,133.746863,318.105187,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
496,model_1_0_3,0.503403,0.323424,0.387132,0.405696,0.397703,0.169066,0.401645,0.190418,0.179918,0.185168,0.207113,0.411176,-0.034577,0.414479,133.554934,317.913258,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
497,model_1_0_2,0.281491,0.106260,0.036235,0.212232,0.125139,0.244615,0.530563,0.299442,0.238487,0.268965,0.258941,0.494586,-0.496893,0.498559,132.816136,317.174460,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
498,model_1_0_1,0.261912,0.098405,0.012530,0.180204,0.097391,0.251281,0.535226,0.306807,0.248183,0.277495,0.261192,0.501280,-0.537683,0.505306,132.762365,317.120689,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
